In [1]:
import os 
import json
from django.contrib.gis.geos import GEOSGeometry
import time 
import shutil 
import glob

import numpy as np
from shapely.geometry import mapping
import pandas as pd
import fiona
import geopandas as gpd
from fiona.crs import to_string
from fiona.crs import from_epsg
from shapely.geometry import shape, Point, Polygon
from fiona.crs import to_string

from madmex.overlay.conversions import train_object_to_feature
from madmex.util.spatial import geometry_transform
from madmex.models import TrainClassification

# 7) Random sample para cada categoría generada de Kmeans

**La muestra debe generarse de modo que incluya los polígonos dados por inecol más otros y que en total sean al menos 1000**

In [2]:
path = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/'
#next line was added because some dc tiles on the first processing pass weren't successful
paths = [
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/46_-35',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/45_-35',
'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/45_-34',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/44_-36',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/43_-35',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/43_-34',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/41_-30',
#'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tilesndvi-gndvi-avi/03-04-2020/40_-34',

]


In [3]:
l=[]

In [4]:
#for source_path in glob.iglob(os.path.join(paths, '/*.shp')):
#    l.append(source_path)
#next line was added because some dc tiles on the first processing pass weren't successful

for path in paths:
    l.append(glob.glob(os.path.join(path, '*.shp'))[0])
        

In [5]:
l_dc_tiles = [elem.split("/")[9] for elem in l]

In [6]:
l_dc_tiles

['45_-34']

In [7]:
filenames = [elem.split("/")[10].replace('aoi2.shp','') for elem in l]

In [8]:
filenames

['03-04-2020_clusters_45_-34_nclusters_4_']

In [9]:
training_set = 'cultivos2019_inecol'

In [10]:
for k in range(len(l)):
    control = 'processing_' + l_dc_tiles[k] + '.txt'
    #file_control = open(path+l_dc_tiles[k]+'/'+control,'w+')
    #next line was added because some dc tiles on the first processing pass weren't successful
    file_control = open(paths[k]+'/'+control,'w+')
    file_control.write('intersection with training set already registered in DB with labels\n')
    gdf_df_clusters = gpd.read_file(l[k])
    crs = gdf_df_clusters.crs
    nclusters=len(gdf_df_clusters['preds'].unique())
    bbox = gdf_df_clusters.total_bounds
    p1 = Point(bbox[0], bbox[3])
    p2 = Point(bbox[2], bbox[3])
    p3 = Point(bbox[2], bbox[1])
    p4 = Point(bbox[0], bbox[1])
    p1 = shape(geometry_transform(mapping(p1),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                                  crs_in=crs))
    p2 = shape(geometry_transform(mapping(p2),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                                  crs_in=crs))
    p3 = shape(geometry_transform(mapping(p3),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                                  crs_in=crs))
    p4 = shape(geometry_transform(mapping(p4),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                                  crs_in=crs))
    np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
    np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
    np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
    np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])
    bb_polygon = Polygon([np1, np2, np3, np4])
    bb_polygon_for_query = GEOSGeometry(json.dumps(mapping(bb_polygon)))
    query_set = TrainClassification.objects.filter(train_object__the_geom__intersects=bb_polygon_for_query,
                                                   training_set=training_set).prefetch_related('train_object', 'interpret_tag')
    fc_cultivos = [train_object_to_feature(x, to_string(crs)) for x in query_set]
    gdf_poi = gpd.GeoDataFrame.from_features([feature for feature in fc_cultivos], crs=crs)
    gdf_df_clusters_poi_intersects = gpd.sjoin(gdf_df_clusters.reset_index(drop=True),
                                               gdf_poi.reset_index(drop=True),
                                               op='intersects',
                                               how='inner')
    file_control.write('finished intersection\n')
    file_control.write('extract rows to keep\n')
    gdf_df_clusters_keep = gpd.GeoDataFrame()
    gdf_df_clusters_keep['features_i'] = gdf_df_clusters_poi_intersects['features_i']
    gdf_df_clusters_keep['preds'] = gdf_df_clusters_poi_intersects['preds']
    gdf_df_clusters_keep['geometry'] = gdf_df_clusters_poi_intersects['geometry']
    gdf_df_clusters_keep['class'] = gdf_df_clusters_poi_intersects['class']
    gdf_df_clusters_keep.crs = crs
    file_control.write('extract sample that are not in the intersection\n')
    t_keep_rows = tuple(gdf_df_clusters_keep['features_i'])
    gdf_df_clusters_to_sample = gdf_df_clusters.loc[~gdf_df_clusters['features_i'].isin(t_keep_rows)]
    nmultipolygons=len(gdf_df_clusters_to_sample.index)
    total_mpolygons = 1000
    if total_mpolygons>len(t_keep_rows):
        nsample=total_mpolygons-len(t_keep_rows)
    else:
        nsample=total_mpolygons   
    if nmultipolygons>nsample:
        nsample_per_cluster=int(nsample/nclusters)+1
        nsample=nsample_per_cluster*nclusters
    else:
        nsample_per_cluster=int(nmultipolygons/nclusters)+1
        nsample=nsample_per_cluster*nclusters
    gdf_df_clusters_sample=gdf_df_clusters_to_sample.groupby('preds').apply(lambda s: s.sample(min(len(s),
                                                                                                    nsample_per_cluster),
                                                                                                random_state=1))
    gdf_df_clusters_sample.reset_index(drop=True,inplace=True)
    
    file_control.write('union of sample and keep polygons\n')
    gdf_sampled_keep = pd.concat([gdf_df_clusters_sample,gdf_df_clusters_keep],
                                 ignore_index=True,sort=False).pipe(gpd.GeoDataFrame)
    if sum(gdf_sampled_keep['class'].isna())>0:
        gdf_sampled_keep.loc[gdf_sampled_keep['class'].isna(),'class'] = ''
    
    file_control.write('write results to shapefile\n')
    total_mpolygons = len(gdf_sampled_keep.index)
    #filename_result = path + l_dc_tiles[k] + '/' + filenames[k] + 'sample_%d' % total_mpolygons + '_with_class_aoi.shp'
    #next line was added because some dc tiles on the first processing pass weren't successful
    filename_result = paths[k] + '/' + filenames[k] + 'sample_%d' % total_mpolygons + '_with_class_aoi.shp'
    gdf_sampled_keep.crs = crs
    gdf_sampled_keep.to_file(filename_result)
    file_control.write('finished processing\n')
    file_control.close()

AttributeError: No geometry data set yet (expected in column 'geometry'.